In [1]:
import tkinter as tk
import time as time
import urllib3 as ur
import time
import threading
import pygame
import socket

pygame 2.0.2 (SDL 2.0.16, Python 3.7.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
http = ur.PoolManager()
pygame.mixer.init()
w = 1
h = 1

#for TV
#w = 1280/1536
#h = 720/864

In [3]:
root = tk.Tk()
ploca1 = tk.Frame(root)


#global lovac, igrac, timer_go, iznos_go, visa, niza, srednja, ponudaNaPloci
global serverSocket
serverSocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM);
serverSocket.bind(("127.0.0.1",9090));
serverSocket.listen();

def pokreniZavrsnu(zavrsna):
    #read = http.request('GET', 'http://192.168.4.1/gamemode?gamemode=zavrsna')
    
    ##globalne varijable
    global igracZav, lovacZav, prav, bodovi, zavrsnaTime, timerText, ide, timerSeq, zavrsnaTimeFinal, zavrsnaTraje, zavrsnaIgrac
    timerSeq = 0
    ide = False
    zavrsnaTraje = True
    zavrsnaTimeFinal = 5
    zavrsnaTime = zavrsnaTimeFinal
    timerText = tk.StringVar()
    timerText.set(str(zavrsnaTime))
    igracZav = 1
    lovacZav = 0
    bodovi = tk.StringVar()
    bodovi.set('1')
    
    #postavljanje kanvasa
    zavrsna.place(x = 0, y = 0)
    
    can = tk.Canvas(zavrsna, width = w*(1536-20), height = h*(564 - 60), bg = 'lightblue', bd = 0,  highlightthickness=0)
    can_up = tk.Canvas(zavrsna, height = h*180, width = w*(1536-20), bg = 'pink', bd = 0,  highlightthickness=0)
    can_down = tk.Canvas(zavrsna, height = h*180, width = w*(1536-20), bg = 'pink', bd = 0,  highlightthickness=0)
    can_up.grid(column = 0, row = 0)
    can.grid(column = 0, row = 1)
    can_down.grid(column=0, row = 2)
    bodoviLab = tk.Label(can, textvariable = bodovi, font = ('Arial', int(w*100)), bg = 'blue') #label na sredini, kaže bodove
    bodoviLab.place(x = w*1536/2, y = h*504/2, anchor = 'center')
    
    prav = []
    prav.append(can.create_rectangle(0, 0, w*1536, h*504, fill = 'blue', width = 5))
    
    zavrsna.tkraise()
    
    def plocaComm():
        zavrsnaTraje = False
        clientSocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM);
        clientSocket.connect(("127.0.0.1",9090)); ##!!!!
        data = 'e';
        clientSocket.send(data.encode());
        dataFromServer = clientSocket.recv(1024);
        zavrsna.destroy()
        ploca2 = tk.Frame(root)
        pokreniPlocu(ploca2)
    
    def readInputFinal():
        global zavrsnaIgrac, serverSocket
        #print('trazim input')
        
        while(zavrsnaTraje == True):
            (clientConnected, clientAddress) = serverSocket.accept();
            dataFromClient = clientConnected.recv(1024)
            data=dataFromClient.decode()    
            zavrsnaIgrac = str(data)[2]
            print(zavrsnaIgrac)
                
            time.sleep(0.4)
            #time.sleep(0.1)
                
                        
        
    
    gumbZaPlocu = tk.Button(zavrsna, text='Ploča', command = plocaComm)  
    gumbZaPlocu.place(x = 40, y = 20)
    
    gumbZaTak = tk.Button(zavrsna, bg = 'pink', command = threading.Thread(target=readInputFinal).start(), 
                          highlightthickness = 0, bd = 0)
    gumbZaTak.place(x = 40, y = 50)
    
    
    def akcija(event):
        c = event.char
        global lovacZav, igracZav, prav, ide
        
        def startZav():
            global zavrsnaTime, timerSeq, timerText, zavrsnaTimeFinal, ide
            if(ide == False):
                return
            timerSeq+= 1
            if(timerSeq == 5):
                timerSeq = 0
                zavrsnaTime -= 1
                timerText.set(str(zavrsnaTime))
            if(zavrsnaTime > 0):
                can_down.after(200, startZav)
                
            else:
                zavrsnaTime = zavrsnaTimeFinal
                timerText.set(str(zavrsnaTime))
                ide = False
                
        #stopZav()
        
        for p in prav:
            can.delete(p)
        if(c == 's'):
            lovacZav += 1
                
        elif(c == 'w' and lovacZav > 0):
            lovacZav -= 1

        elif(c == 'f'):
            igracZav += 1

        elif(c == 'r' and igracZav > 1):
            igracZav -= 1
        elif(c == ' '):
            if(ide == False):
                ide = True
                startZav()    
            else:
                ide = False
                #stopZav()
            #print('spejs')
        
        prav = []
        for i in range(0, igracZav):
            d = (1536)/igracZav
            prav.append(can.create_rectangle(w*i*d, 0, w*(i+1)*d, h*504, fill = 'blue', width = 5))
        for i in range(0, lovacZav):
            can.itemconfig(prav[i], fill='gold')
        
        bodovi.set(igracZav)
        bodoviLab.place(x = (1536)/igracZav*(igracZav - 1 + 1/2)*w, y = h*504/2, anchor = 'center')
        
        if(igracZav == lovacZav):
            bodovi.set('')
            bodoviLab.config(bg = 'gold')
        if(igracZav == 10):
            bodoviLab.config(font = ('Arial', int(w*75)))
        elif(igracZav == 9):
            bodoviLab.config(font=('Arial', int(w*100)))
        elif(igracZav == 13):
            bodoviLab.config(font = ('Arial', int(w*50)))
        
        
    timerLabel = tk.Label(can_down, textvariable = timerText, font=('Arial', 100), bg = 'pink')
    timerLabel.place(x = w*1536/2, y = h*100, anchor = 'center')
            
    
    root.bind("<Key>", akcija)
    

In [4]:
def pokreniPlocu(ploca):
    #postavljanje novog okvira
    ploca.place(x = 0, y = 0)
    ploca.tkraise()
    
    #globalne varijable
    global lovac, igrac, timer_go, iznos_go, visa, niza, srednja, ponudaNaPloci
    global lovacStisnuoText, igracStisnuoText, plocaTraje
    #global serverSocket
    #serverSocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM);
    #serverSocket.bind(("127.0.0.1",9090));
    #serverSocket.listen();
    
    plocaTraje = True   #treba onom threadu da zna kad zaustaviti proces
    lovac = 0
    igrac = 0
    timer_go = 0
    iznos_go = 0
    iznos_value = tk.StringVar()
    iznos_pomoc = tk.StringVar()
    iznos_pomoc.set('Srednja ponuda:')
    visa = tk.StringVar()
    niza = tk.StringVar()
    srednja = tk.StringVar()
    visa.set('Viša: ')
    niza.set('Niža: ')
    srednja.set('Srednja: ')
    ponudaNaPloci = tk.StringVar()
    ponudaNaPloci.set('')
    lovacStisnuoText=tk.StringVar(); lovacStisnuoText.set('')
    igracStisnuoText=tk.StringVar(); igracStisnuoText.set('')

    
    
    #pygame.mixer.music.load()  #loada muziku za kliknuti gumb
    

    
    ###novi thread koji se bavi inputom od arduina
    def readInputArduino():
        global lovacStisnuoText, igracStisnuoText, serverSocket
        #serverSocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM);
        #serverSocket.bind(("127.0.0.1",9090));
        #serverSocket.listen();

        while(plocaTraje == True):
            
            (clientConnected, clientAddress) = serverSocket.accept();
            dataFromClient = clientConnected.recv(1024)
            
            data=dataFromClient.decode()
            dataStr = str(data)
            if(dataStr[0] == '9'):
                #print('kraj ovoga')
                clientConnected.send('ok'.encode())
                break
            
            
            lovacArduinoData = '0'
            igracArduinoData = '0'
            if(dataStr[2] == '1' or dataStr[2] == '2' or dataStr[2] =='3'):
                lovacArduinoData = dataStr[2]
            elif(dataStr[2] == '4' or dataStr[2] == '5' or dataStr[2] =='6'):
                igracArduinoData = dataStr[2]
            elif(dataStr[2]=='a'):
                lovacArduinoData = '1'; igracArduinoData = '4'
            elif(dataStr[2]=='b'):
                lovacArduinoData = '2'; igracArduinoData = '4'
            elif(dataStr[2]=='c'):
                lovacArduinoData = '3'; igracArduinoData = '4'
            elif(dataStr[2]=='d'):
                lovacArduinoData = '1'; igracArduinoData = '5'
            elif(dataStr[2]=='e'):
                lovacArduinoData = '2'; igracArduinoData = '5'
            elif(dataStr[2]=='f'):
                lovacArduinoData = '3'; igracArduinoData = '5'
            elif(dataStr[2]=='g'):
                lovacArduinoData = '1'; igracArduinoData = '6'
            elif(dataStr[2]=='h'):
                lovacArduinoData = '2'; igracArduinoData = '6'
            elif(dataStr[2]=='i'):
                lovacArduinoData = '3'; igracArduinoData = '6'
            
            s1 = pygame.mixer.Sound("D://Users//Šimun//Music//mjuza//potjera//ploca-start.wav")
            s2 = pygame.mixer.Sound("D://Users//Šimun//Music//mjuza//potjera//ploca-clock.wav")
            pygame.mixer.Channel(0).play(s1)
        
            if(lovacArduinoData == '0'): 
                pygame.mixer.Channel(1).play(s2)
                #pygame.mixer.music.play(loops = 0)
                #pygame.mixer.music.load("D://Users//Šimun//Music//mjuza//potjera//ploca-clock.wav")
                #pygame.mixer.music.play(loops=0)
                currentTime = time.time()
                while(time.time() - currentTime < 5):
                    (clientConnected, clientAddress) = serverSocket.accept();
                    dataFromClient = clientConnected.recv(1024)
                    data=dataFromClient.decode()
                    dataStr = str(data)
                    if(dataStr[2] == '1' or dataStr[2] == '2' or dataStr[2] =='3'):
                        lovacArduinoData = dataStr[2]
                    if(lovacArduinoData != '0'):
                        break
                    time.sleep(0.1)  
                pygame.mixer.Channel(1).stop()
            elif(igracArduinoData == '0'):  
                pygame.mixer.Channel(1).play(s2)
                #pygame.mixer.music.play(loops = 0)
                #pygame.mixer.music.load("D://Users//Šimun//Music//mjuza//potjera//ploca-clock.wav")
                #pygame.mixer.music.play(loops=0)
                currentTime = time.time()
                while(time.time() - currentTime < 5):
                    (clientConnected, clientAddress) = serverSocket.accept();
                    dataFromClient = clientConnected.recv(1024)
                    data=dataFromClient.decode()
                    dataStr = str(data)
                    if(dataStr[2] == '4' or dataStr[2] == '5' or dataStr[2] =='6'):
                        igracArduinoData = dataStr[2]
                    if(igracArduinoData != '0'):
                        break
                    time.sleep(0.1)  #bilo je 0.1 pa 0.05
                pygame.mixer.Channel(1).stop()
            if(igracArduinoData != '0' and lovacArduinoData != ' 0'):
                pygame.mixer.music.load("D://Users//Šimun//Music//mjuza//potjera//ploca-end.wav")
                pygame.mixer.music.play(loops = 0)
            lovacStisnuoText.set(chr(ord('A') - 1 + int(lovacArduinoData)))
            igracStisnuoText.set(chr(ord('A') - 4 + int(igracArduinoData)))  #!!! promjena !!!
                
            time.sleep(1)    
            
            time.sleep(0.1)
        #print('aaa')
        
            
        
    
    
    #kod za brzopotezna - timer
    def brza():
        global vrij, ide
        pygame.mixer.music.load("D://Users//Šimun//Music//mjuza//potjera//brzopot-end.wav")  #loada zvuk za kraj brzopotezne
        vrij = 5 #broj sekundi u brzopoteznoj
        ide = False
        prozor = tk.Toplevel(root, height = 2500, width = 2500)
        prozor.lift()
        temp_can = tk.Canvas(prozor, width = 600, height = 600)
        temp_can.grid(row = 0, column = 0)
        temp_str = tk.StringVar()
        temp_str.set(str(vrij))

        def klik():
            global ide
            if(ide == False):
                ide = True
                start()
            else:
                #ide = False
                stop()

        def start():
            #print(2)
            global vrij, ide, timer_go
            if(ide == False):
                return
            timer_go += 1
            if(timer_go == 5):
                vrij -= 1
                temp_str.set(str(vrij))
                timer_go = 0
                
            if(vrij > 0):
                temp_can.after(200, start)
            else:
                pygame.mixer.music.play(loops = 0)  #tu još treba postaviti na početni link

        def stop():
            global vrij, ide
            ide = False
            vrij += 1


        broj = tk.Label(temp_can, textvariable = temp_str, font = ('Arial', 95))
        broj.place(x = 250, y = 100)
        gumb = tk.Button(temp_can, text = "Start/Stop", command = klik, font = ('Arial', 20))
        gumb.place(x = 200, y = 400)


    #kod za pomicanje igrača i lovca
    def akcija(event):   
        c = event.char
        global lovac
        global igrac
        if(c == 's'):
            if(lovac < 7):
                lovac += 1
                can.itemconfig(prav[lovac-1], fill='red')
                can.move(trokutLovac, 0, h*600/7)
                if(igrac + 1 == lovac):
                    can.move(trokutIgrac1, 0, h*600/7)
                    can.move(trokutIgrac2, 0, h*600/7)
                    can.itemconfig(trokutIgrac1, fill='darkblue')
                    can.itemconfig(trokutIgrac2, fill='darkblue')
                    ponuda.place(y = h*(600/7/2 + (1+igrac)*600/7), x = w*(1536 - 700)/2, anchor = 'center')
                    ponudaNaPloci.set('')

        elif(c == 'w'):
            if(lovac != 0):
                lovac -= 1
                can.itemconfig(prav[lovac], fill='blue')
                can.move(trokutLovac, 0, -h*600/7)

        elif(c == 'f'):
            if(igrac < 7):
                igrac += 1
                can.itemconfig(prav[igrac-1], fill='blue')
                ponuda.place(y = h*(600/7/2 + igrac*600/7), x = w*(1536 - 700)/2, anchor = 'center')
                #ponuda.place(x = 700, y = 10+600/7*igrac)
                can.move(trokutIgrac1, 0, h*600/7)
                can.move(trokutIgrac2, 0, h*600/7)


        elif(c == 'r'):
            if(igrac != 0):
                igrac -= 1
                can.itemconfig(prav[igrac], fill='darkblue')
                ponuda.place(y = h*(600/7/2 + igrac*600/7), x = w*(1536 - 700)/2, anchor = 'center')
                #ponuda.place(x = 700, y = 10+600/7*igrac)
                can.move(trokutIgrac1, 0, -h*600/7)
                can.move(trokutIgrac2, 0, -h*600/7)



    #kod za namještanje iznosa ponuda
    def iznosCommand(event):   
        global iznos_go, niza, visa, srednja
        if(iznos_go == 0):
            iznos_pomoc.set('Više ponuda:')
            iznos_go = 1
            srednja.set('Srednja: ' + iznos_value.get())
        elif(iznos_go == 1):
            iznos_pomoc.set('Niža ponuda:')
            iznos_go = 2
            visa.set('Viša: ' + iznos_value.get())
        else:
            iznos_pomoc.set('Srednja ponuda:')
            iznos_go = 0
            niza.set('Niža: ' + iznos_value.get())
        iznos_value.set('')
        #print(niza.get(), visa.get(), srednja.get())


    def uzmiSredCom():
        ponudaNaPloci.set(srednja.get().replace('Srednja: ', ''))
        global igrac, iznos_go
        iznos_go = 0
        iznos_pomoc.set('Srednja ponuda:')
        niza.set('Niža: ')
        srednja.set('Srednja: ')
        visa.set('Viša: ')
        igrac = 2
        for i in range(0, igrac):
            can.itemconfig(prav[i], fill='blue')
        ponuda.place(y = h*(600/7/2 + igrac*600/7), x = w*(1536 - 700)/2, anchor = 'center')
        can.itemconfig(trokutIgrac1, fill = 'white')
        can.coords(trokutIgrac1, w*20, h*(600/7 + 5), w*20, h*(600/7*2 - 5), w*40, h*(600/7/2 + 600/7))
        can.coords(trokutIgrac2, w*(1536 - 700 - 20), h*(600/7 + 5), w*(1536 - 700 - 20), h*(600/7*2 - 5),
                   w*(1536 - 700 - 40), h*(600/7 + 600/7/2))
        can.move(trokutIgrac1, 0, h*600/7)
        can.itemconfig(trokutIgrac2, fill = 'white')
        can.move(trokutIgrac2, 0, h*600/7)
        can.itemconfig(trokutLovac, fill = 'black')

    def uzmiNizCom():
        ponudaNaPloci.set(niza.get().replace('Niža: ', ''))
        global igrac, iznos_go
        iznos_go = 0
        iznos_pomoc.set('Srednja ponuda:')
        niza.set('Niža: ')
        srednja.set('Srednja: ')
        visa.set('Viša: ')
        igrac = 3
        for i in range(0, igrac):
            can.itemconfig(prav[i], fill='blue')
        ponuda.place(y = h*(600/7/2 + igrac*600/7), x = w*(1536 - 700)/2, anchor = 'center')
        can.itemconfig(trokutIgrac1, fill = 'white')
        can.itemconfig(trokutIgrac2, fill = 'white')
        can.coords(trokutIgrac1, w*20, h*(600/7 + 5), w*20, h*(600/7*2 - 5), w*40, h*(600/7/2 + 600/7))
        can.coords(trokutIgrac2, w*(1536 - 700 - 20), h*(600/7 + 5), w*(1536 - 700 - 20), 
                   h*(600/7*2 - 5), w*(1536 - 700 - 40), h*(600/7 + 600/7/2))
        can.move(trokutIgrac1, 0, h*600/7*2)
        can.move(trokutIgrac2, 0, h*600/7*2)
        can.itemconfig(trokutLovac, fill = 'black')


    def uzmiVisCom():
        ponudaNaPloci.set(visa.get().replace('Viša: ', ''))
        global igrac, iznos_go
        iznos_go = 0
        iznos_pomoc.set('Srednja ponuda:')
        niza.set('Niža: ')
        srednja.set('Srednja: ')
        visa.set('Viša: ')
        igrac = 1
        for i in range(0, igrac):
            can.itemconfig(prav[i], fill='blue')
        ponuda.place(y = h*(600/7/2 + igrac*600/7), x = w*(1536 - 700)/2, anchor = 'center')
        can.coords(trokutIgrac1, w*20, h*(600/7 + 5), w*20, h*(600/7*2 - 5), w*40, h*(600/7/2 + 600/7))
        can.coords(trokutIgrac2, w*(1536 - 700 - 20), h*(600/7 + 5), w*(1536 - 700 - 20),
                   h*(600/7*2 - 5), w*(1536 - 700 - 40), h*(600/7 + 600/7/2))
        can.itemconfig(trokutIgrac1, fill = 'white')
        can.itemconfig(trokutIgrac2, fill = 'white')
        can.itemconfig(trokutLovac, fill = 'black')

    #ovaj dio je dodan tak da kad upišeš sve tri ponude, ne pišeš gluposti u onaj prozor
    def focus(event):
        gumbZaTak.focus()  

    #funkcija za resertiranje ploče
    def resetCom():
        global lovac, igrac, timer_go, iznos_go, visa, niza, srednja, ponudaNaPloci
        can.move(trokutLovac, 0, -h*600/7*lovac)
        can.coords(trokutIgrac1, w*20, h*(600/7 + 5), w*20, h*(600/7*2 - 5), w*40, h*(600/7/2 + 600/7))
        can.coords(trokutIgrac2, w*(1536 - 700 - 20), h*(600/7 + 5), w*(1536 - 700 - 20), h*(600/7*2 - 5),
                   w*(1536 - 700 - 40), h*(600/7 + 600/7/2))
        can.itemconfig(trokutIgrac1, fill = 'darkblue')
        can.itemconfig(trokutIgrac2, fill = 'darkblue')
        lovac = 0
        igrac = 0
        iznos_go = 0
        iznos_pomoc.set('Srednja ponuda:')
        visa.set('Viša: ')
        niza.set('Niža: ')
        srednja.set('Srednja: ')
        ponudaNaPloci.set('')
        for i in range(0, 7):
            can.itemconfig(prav[i], fill='darkblue')

    #fja za mijenjanje game-moda
    def zavrsnaCom():
        global plocaTraje#, serverSocket
        plocaTraje = False
        #time.sleep(0.5)
        clientSocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM);
        clientSocket.connect(("127.0.0.1",9090));
        data = '9';
        clientSocket.send(data.encode());
        dataFromServer = clientSocket.recv(1024);
        ploca.destroy()
        zavrsna2 = tk.Frame(root)
        pokreniZavrsnu(zavrsna2)

    #povećava na cijeli ekran
    root.attributes("-fullscreen", True)
    root.bind("<Key>", akcija)


    #izrada grafičkog sučelja - tu bude možda trebalo namještat za rad na televizoru
    can = tk.Canvas(ploca, width = w*(1536 - 700), height = h*600, bg = 'blue', bd = 0,  highlightthickness=0)
    can.grid(row = 0, column = 1)

    prav = []
    for i in range(0, 7):
        d = 600/7
        prav.append(can.create_rectangle(0, h*i*d, w*(1536-700-1), h*(i*d + d), fill = 'darkblue'))


    ponuda = tk.Label(can, textvariable = ponudaNaPloci, bg = 'darkblue', font = ('Arial', 25), fg = 'white')
    ponuda.place(y = h*600/7/2, x = w*(1536 - 700)/2, anchor = 'center')

    trokutLovac = can.create_polygon(w*20, h*10, w*(1536 - 700 - 20), h*10, w*(1536 - 700)/2, h*50, fill = 'darkblue')
    trokutIgrac1 = can.create_polygon(w*20, h*(600/7 + 5), w*20, h*(600/7*2 - 5), w*40, h*(600/7/2 + 600/7), fill = 'darkblue')
    trokutIgrac2 = can.create_polygon(w*(1536 - 700 - 20), h*(600/7 + 5), w*(1536 - 700 - 20), h*(600/7*2 - 5), 
                                      w*(1536 - 700 - 40), h*(600/7 + 600/7/2), fill = 'darkblue')
    can.move(trokutLovac, 0, -h*600/7)

    can_left = tk.Canvas(ploca, width = w*350, height = h*600, bg = 'pink', bd = 0,  highlightthickness=0)
    can_left.grid(row = 0, column = 0)

    can_right = tk.Canvas(ploca, width = w*350, height = h*600, bg = 'pink',  highlightthickness=0)
    can_right.grid(row = 0, column = 2)



    #lijevi dio ploče - gumbi
    timer = tk.Button(ploca, text='Brzopotezna', bg = 'black', fg = 'white', command=brza, width = int(w*20))
    timer.place(x = w*50, y = h*50)

    gumbZaTak = tk.Label(ploca, bg = 'pink')
    gumbZaTak.place(x = w*50, y = h*450)  

    gumbZaTak2 = tk.Label(ploca, bg = 'pink', command = threading.Thread(target=readInputArduino).start())
    gumbZaTak2.place(x = w*50, y = h*500)
    
    final = tk.Button(ploca, text = 'Završna', bg = 'black', fg = 'white', width = int(w*20), command = zavrsnaCom)
    final.place(x = w*50, y = h*100) 

    reset = tk.Button(ploca, text = 'Reset', bg = 'black', fg = 'white', width = int(w*20), command = resetCom)
    reset.place(x = w*50, y = h*500)  

    #desni dio ploče - ponude
    iznosLabel = tk.Label(ploca, textvariable = iznos_pomoc, bg = 'pink')
    iznosLabel.place(x = w*(350 + 1536 - 700 + 100), y = h*50)
    unos_iznos = tk.Entry(ploca, textvariable = iznos_value)
    unos_iznos.bind("<Return>", iznosCommand)
    unos_iznos.place(y = h*100, x = w*(350 + 1536 -700 + 100))

    visaLabel = tk.Label(ploca, textvariable = visa, bg = 'pink', font = ('Arial', 15))
    srednjaLabel = tk.Label(ploca, textvariable = srednja, bg = 'pink', font = ('Arial', 15))
    nizaLabel = tk.Label(ploca, textvariable = niza, bg = 'pink', font = ('Arial', 15))
    visaLabel.place(y = h*200, x = w*(350 + 1536 -700 + 120))
    srednjaLabel.place(y = h*250, x = w*(350 + 1536 -700 + 120))
    nizaLabel.place(y = h*300, x = w*(350 + 1536 -700 + 120))

    uzmiSred = tk.Button(ploca, text = 'SREDNJA', bg = 'black', fg = 'white', width = int(w*6), command = uzmiSredCom)
    uzmiVis = tk.Button(ploca, text = 'VIŠA', bg = 'black', fg = 'white', width = int(w*6), command = uzmiVisCom)
    uzmiNiz = tk.Button(ploca, text = 'NIŽA', bg = 'black', fg = 'white', width = int(w*6), command = uzmiNizCom)
    uzmiSred.place(y = h*255, x = w*(350 + 1536 - 700 + 30))
    uzmiVis.place(y = h*205, x = w*(350 + 1536 - 700 + 30))
    uzmiNiz.place(y = h*305, x = w*(350 + 1536 - 700 + 30))

    can_right.bind("<Button>", focus)  


    #donji kanvasi
    can_down1 = tk.Canvas(ploca, width = w*350, height = h*264, bg = 'purple', bd = 0,  highlightthickness=0)
    can_down1.grid(row = 1, column = 0)
    
    
    
    can_down2 = tk.Canvas(ploca, width = w*(1536 - 700), height = h*264, bg = 'purple', bd = 0,  highlightthickness=0)
    can_down2.grid(row = 1, column = 1)

    can_down3 = tk.Canvas(ploca, width = w*350, height = h*264, bg = 'purple', bd = 0,  highlightthickness=0)
    can_down3.grid(row = 1, column = 2)

    lovacStisnuo = tk.Label(ploca, textvariable=lovacStisnuoText, font = ('Arial', 50), anchor='center', bg = 'purple', 
                            fg = 'white')
    lovacStisnuo.place(x = w*350, y = h*(600+132-50))
    
    igracStisnuo = tk.Label(ploca, textvariable = igracStisnuoText, font = ('Arial', 50), anchor = 'center', bg = 'purple',
                           fg = 'white')
    igracStisnuo.place(x = w*(1536-700+350), y = h*(600+132-50))




    

In [5]:
pokreniPlocu(ploca1)  #pokreće kod, počevši s pločom
root.mainloop() #pokreće aplikaciju

Exception in thread Thread-6:
Traceback (most recent call last):
  File "D:\Anaconda\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "D:\Anaconda\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Šimun\AppData\Local\Temp\ipykernel_14072\1872007162.py", line 64, in readInputFinal
    zavrsnaIgrac = str(data)[2]
IndexError: string index out of range

